In [1]:
import numpy as np
import pandas as pd
from scipy import stats


import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import scipy.stats as scs
import statsmodels.stats as sms

import matplotlib.pyplot as plt
import matplotlib as mpl


# 引入Kalman函数库
from pykalman import KalmanFilter

# 引入纯随机序列检测，只有是非纯随机序列，才能进行检测
from statsmodels.stats.diagnostic import acorr_ljungbox

# 引入时序分析模型进行模型匹配
from statsmodels.tsa.arima_model import ARMA

import itertools

from jqdata import *
from jqfactor import *

# 研究中建立回测（需要的策略中提前完成策略编写）
通过设置不同的参数后建立回测数据，通过回测数据计算因子载荷

In [5]:
#需要自己设置的参数,False/True参数传过去显示没法序列化，但是直接用{}传过去没问题

paramNames=['factor_num','stocks_num']
paramList=[[4,5,6],[30,50,70,90]]
# paramList=[[3],[30]]

algorithmID='9fa65d9ae8bd261f3fac98f05d81e86e'

startDate='2015-01-01'
endDate='2016-12-31'
frequency='day'
initialCash=1000000
maxBacktestNum=9 #最大同时回测数

# 分解参数列表，将每个参数进行映射
params=list(itertools.product(*paramList))

# 存储结果的数组
df = pd.DataFrame(params,index=params,columns=paramNames)

# backtest的存储列表
created_bt_ids = []


for i in range(min(maxBacktestNum,df.shape[0])):
    var = df.iloc[i].to_dict()
    out_algorithm_id = create_backtest(algorithm_id=algorithmID, # id 需要自己拷贝
                        start_date=startDate, 
                        end_date=endDate, 
                        frequency=frequency, 
                        initial_cash=initialCash, 
                        initial_positions=None, 
                        extras= eval(str(var)),
                        name=str(var)
                        )
    
    gt = get_backtest(out_algorithm_id)
    
    while gt.get_status() != 'done':
        time.sleep(5)
        created_bt_ids.append(out_algorithm_id)
    


In [6]:
# backtest_id是所有回测的ID

# 存储最终的结果
res_backtest = {}

#填入不同参数下的收益数据
for i in range(len(created_bt_ids)):
    res = {}
    
    gt = get_backtest(backtest_id = created_bt_ids[i])

    
    # 获得名称
    name = gt.get_params()['name'] 
    res_backtest[name] = []
    
    # 获得基本的参数
    res = gt.get_risk()
    
    res_backtest[name].append(res['algorithm_return'])
    res_backtest[name].append(res['max_drawdown'])



In [7]:
pd.DataFrame.from_dict(res_backtest, orient='index')

,0,1
"&#123;&#39;factor_num&#39;: 4, &#39;stocks_num&#39;: 30}",0.503906,0.423193
"&#123;&#39;factor_num&#39;: 4, &#39;stocks_num&#39;: 50}",0.277746,0.446062
"&#123;&#39;factor_num&#39;: 4, &#39;stocks_num&#39;: 70}",0.236117,0.453804
"&#123;&#39;factor_num&#39;: 4, &#39;stocks_num&#39;: 90}",0.257764,0.448384
"&#123;&#39;factor_num&#39;: 5, &#39;stocks_num&#39;: 30}",0.456989,0.438509
"&#123;&#39;factor_num&#39;: 5, &#39;stocks_num&#39;: 50}",0.314244,0.438353
"&#123;&#39;factor_num&#39;: 5, &#39;stocks_num&#39;: 70}",0.225764,0.459334
"&#123;&#39;factor_num&#39;: 5, &#39;stocks_num&#39;: 90}",0.193378,0.467779
"&#123;&#39;factor_num&#39;: 6, &#39;stocks_num&#39;: 30}",0.485910,0.412235


In [5]:
pd.DataFrame.from_dict(res_backtest, orient='index').to_csv('test1.csv')

# 根据回测获取基本信息


In [13]:
# 填写algorithm ID
gt = get_backtest(backtest_id = '751eb71519082465d3f47518c4aa315f' )

# 获取结果
res = gt.get_results()

# 显示结果
gt.get_risk()

{'__version': 101,
 'algorithm_return': 0.0753598444950001,
 'algorithm_volatility': 0.206290540882483,
 'alpha': -0.032821074348489215,
 'annual_algo_return': 0.027648340479597744,
 'annual_bm_return': 0.06360818074638397,
 'avg_excess_return': -0.00010142684845106405,
 'avg_position_days': 60.3421052631579,
 'avg_trade_return': 0.016094722308521065,
 'benchmark_return': 0.178545533642694,
 'benchmark_volatility': 0.18320956649284728,
 'beta': 0.867047531022577,
 'day_win_ratio': 0.5225225225225225,
 'excess_return': -0.08755341749822865,
 'excess_return_max_drawdown': 0.25529284970166977,
 'excess_return_max_drawdown_period': ['2017-04-11', '2018-02-06'],
 'excess_return_sharpe': -0.5494656006279569,
 'information': -0.26866137536453666,
 'lose_count': 279,
 'max_drawdown': 0.3274010879545035,
 'max_drawdown_period': ['2017-09-06', '2018-10-18'],
 'max_leverage': 0.0,
 'period_label': '2019-09',
 'profit_loss_ratio': 1.090412568596583,
 'sharpe': -0.059875064884524175,
 'sortino': -0

# 显示行业总共有哪些股票


In [9]:
# 提取行业的名称
industry_list = get_industries(name='sw_l1', date='2019-5-17').index
industry_name = get_industries(name='sw_l1', date='2019-5-17')['name']

industry_dict = {}

for i in range(0,len(industry_list)):
    # 获取行业编码列表
    industry_i =  industry_list[i]
    
    # 根据行业编码提取相应的股票
    sec_pool = []
    sec_pool = get_industry_stocks(industry_i,date='2019-5-17')

    # 赋值
    industry_dict[industry_name[i]] = len(sec_pool)
    
    

pd.DataFrame.from_dict(industry_dict,orient='index').shape

# pd.DataFrame.from_dict(industry_dict,orient='index')
    

#     Secs_Chosen = []
#     Secs_Chosen = Sec_Choose(context,sec_pool)
#     print "选出股票：%s"%str(Secs_Chosen)

#     candidate_list =  candidate_list + Secs_Chosen
# print(get_industries(name='zjw', date='2019-5-17'))
# print(get_industries(name='zjw', date='2019-5-17').shape)


(28, 1)

In [11]:
res_backtest

{'&#123;&#39;factor_num&#39;: 3, &#39;stocks_num&#39;: 30}': [0.2903464259200039,
  0.4406520507573568]}

In [12]:
pd.DataFrame(res_backtest)

,"&#123;&#39;factor_num&#39;: 3, &#39;stocks_num&#39;: 30}"
0,0.290346
1,0.440652


In [13]:
pd.DataFrame.from_dict(res_backtest, orient='index')

,0,1
"&#123;&#39;factor_num&#39;: 3, &#39;stocks_num&#39;: 30}",0.290346,0.440652


In [ ]:
#先获取基准收益和交易日信息
gt = get_backtest(backtest_id = created_bt_ids[0])
# 结果
res = gt.get_results()

# 基准收益率
b_return = []
# 时间
t = []

# 基准收益率存储
for r in res:
    b_return.append(r['benchmark_returns'])
    t.append(r['time'])

#建立df存储数据   
factor_data_pd = pd.DataFrame(index = t)
factor_data_pd['b_return'] = b_return

#填入不同参数下的收益数据
for i in range(len(created_bt_ids)):
    gt = get_backtest(backtest_id = created_bt_ids[i])
    res = gt.get_results()
    name = gt.get_params()['name'] 
    s_return = []
    for r in res:
        s_return.append(r['returns'])

    factor_data_pd[name] = s_return

factor_data_pd.plot(figsize = (15,10))

In [ ]:
factor_data_pd

In [9]:
factor_data_pd.to_csv("20190924.csv")

In [4]:
#先获取基准收益和交易日信息
gt = get_backtest(backtest_id = created_bt_ids[0])
# 结果
res = gt.get_results()

gt.get_risk()

{'__version': 101,
 'algorithm_return': 0.2903464259200039,
 'algorithm_volatility': 0.360622029372603,
 'alpha': 0.17901489072853866,
 'annual_algo_return': 0.1394999354855091,
 'annual_bm_return': -0.032935736415346306,
 'avg_excess_return': 0.0006773238407443955,
 'avg_position_days': 91.04819277108433,
 'avg_trade_return': 0.049304101658580644,
 'benchmark_return': -0.06328211223363389,
 'benchmark_volatility': 0.3185058080061141,
 'beta': 1.0902056954661656,
 'day_win_ratio': 0.5614754098360656,
 'excess_return': 0.37751872017398913,
 'excess_return_max_drawdown': 0.0969767242486419,
 'excess_return_max_drawdown_period': ['2015-06-29', '2015-09-25'],
 'excess_return_sharpe': 1.3567517849630504,
 'information': 1.69903495929358,
 'lose_count': 121,
 'max_drawdown': 0.4406520507573568,
 'max_drawdown_period': ['2015-06-12', '2015-08-26'],
 'max_leverage': 0.0,
 'period_label': '2016-12',
 'profit_loss_ratio': 1.470935927585358,
 'sharpe': 0.2759119725952833,
 'sortino': 0.3178109365